In [17]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample conversation data (input-output pairs)
conversations = {
    "What is 2 plus 2?": "The answer is 4.",
    "How do you run Python code?": "You can run Python code using the 'python' command.",
    "Can you translate hello to Spanish?": "The translation of 'hello' to Spanish is 'hola'.",
    "What is the sum of 5 and 7?": "The answer is 12.",
    "Run this Python code": "Code executed successfully."
}

input_texts = list(conversations.keys())
target_texts = list(conversations.values())

# Tokenization
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(input_texts + target_texts)
word_index = tokenizer.word_index

# Convert text to sequences
input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

# Padding sequences
input_padded = pad_sequences(input_sequences, padding='post')
target_padded = pad_sequences(target_sequences, padding='post')

vocab_size = len(word_index) + 1
max_sequence_len = input_padded.shape[1]

# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_sequence_len),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(vocab_size, activation='softmax')  # Predict the next word
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prepare target labels as the next word in the sequence
target_labels = [sequence[1:] for sequence in target_sequences]  # Skip the first word for each target
target_labels = pad_sequences(target_labels, maxlen=max_sequence_len, padding='post')

# Convert target labels to a flat array (to match model output shape)
target_labels = np.array([label[0] for label in target_labels])  # Just use the first word for simplicity

# Train the model
model.fit(input_padded, target_labels, epochs=2000, verbose=1)


Epoch 1/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 3.5568
Epoch 2/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6000 - loss: 3.5388
Epoch 3/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6000 - loss: 3.5225
Epoch 4/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6000 - loss: 3.5057
Epoch 5/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6000 - loss: 3.4880
Epoch 6/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6000 - loss: 3.4684
Epoch 7/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6000 - loss: 3.4465
Epoch 8/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6000 - loss: 3.4221
Epoch 9/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6000 - loss: 3.3948
Epoch 10/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4000 - loss: 3.3639
Epoch 11/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4000 - loss: 3.3284
Epoch 12/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accurac

In [19]:
def generate_response(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_len, padding='post')
    prediction = model.predict(padded_sequence)
    
    # Convert the predicted sequence back to text
    predicted_word_index = np.argmax(prediction, axis=-1)[0]
    predicted_word = tokenizer.index_word.get(predicted_word_index, "")
    
    return predicted_word

# Test the chatbot
response = generate_response("What is 2 plus 2?")
print(f"Chatbot: {response}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Chatbot: answer


In [20]:
model.save('chatbot_model.h5')


In [22]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('chatbot_model.h5')

# Generate a response
response = generate_response("How do you run Python code?")
print(f"Chatbot: {response}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
Chatbot: can
